## European Gas Pipeline Knowledge Graph

In [1]:
!pip install -q neo4j

In [2]:
from neo4j import GraphDatabase
import time
import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', 0)

In [3]:
class Neo4jConnection:
    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)

    def close(self):
        if self.__driver is not None:
            self.__driver.close()

    def query(self, query, parameters=None, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try:
            session = self.__driver.session(database=db) if db is not None else self.__driver.session()
            response = list(session.run(query, parameters))
        except Exception as e:
            print("Query failed:", e)
        finally:
            if session is not None:
                session.close()
        return response

In [4]:
uri  = 'bolt://localhost:7687'
pwd  = 'graph4va'
user = 'neo4j'
conn = Neo4jConnection(uri=uri, user=user , pwd=pwd)

In [5]:
# creating constraints on graph

conn.query("CREATE CONSTRAINT BorderPoints IF NOT EXISTS ON (n:BorderPoint) ASSERT n.id IS UNIQUE;")
conn.query("CREATE CONSTRAINT Compressors IF NOT EXISTS ON (n:Compressor) ASSERT n.id IS UNIQUE;")
conn.query("CREATE CONSTRAINT Consumers IF NOT EXISTS ON (n:Consumer) ASSERT n.id IS UNIQUE;")
conn.query("CREATE CONSTRAINT LNGs IF NOT EXISTS ON (n:LNG) ASSERT n.id IS UNIQUE;")
conn.query("CREATE CONSTRAINT Nodes IF NOT EXISTS ON (n:Node) ASSERT n.id IS UNIQUE;")
conn.query("CREATE CONSTRAINT PowerPlants IF NOT EXISTS ON (n:PowerPlant) ASSERT n.id IS UNIQUE;")
conn.query("CREATE CONSTRAINT Productions IF NOT EXISTS ON (n:Production) ASSERT n.id IS UNIQUE;")
conn.query("CREATE CONSTRAINT Storages IF NOT EXISTS ON (n:Storage) ASSERT n.id IS UNIQUE;")

[]

## Component Creation

In [6]:
# border points

conn.query("""
LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/iamvarol/blogposts/main/medium/europe_gas_network/data/IGGIELGNC3_BorderPoints.csv' as row
FIELDTERMINATOR ';'
WITH row.id as id,
     replace(row.name, '"','') as name,
     row.country_code as country_code,
     apoc.convert.fromJsonList(row.source_id) as source_ids,
     apoc.convert.fromJsonList(row.node_id) as node_ids,
     point({latitude:toFloat(row.lat), longitude:toFloat(row.long)}) as loc,
     apoc.convert.fromJsonMap(replace(row.param, 'None', 'null')) as params
MERGE(b:BorderPoint {id:id})
ON CREATE SET b.name          = name,
              b.country_code  = country_code,
              b.source_id     = source_ids[0],
              b.node_id       = node_ids[0],
              b.loc           = loc,
              b              += params
""")

[]

In [7]:
# compressors

conn.query("""
LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/iamvarol/blogposts/main/medium/europe_gas_network/data/IGGIELGNC3_Compressors.csv' as row
FIELDTERMINATOR ';'
WITH row.id as id,
     replace(row.name, '"','') as name,
     row.country_code as country_code,
     apoc.convert.fromJsonList(row.source_id) as source_ids,
     apoc.convert.fromJsonList(row.node_id) as node_ids,
     point({latitude:toFloat(row.lat), longitude:toFloat(row.long)}) as loc,
     apoc.convert.fromJsonMap(replace(row.param, 'None', 'null')) as params
MERGE(c:Compressor {id:id})
ON CREATE SET c.name          = name,
              c.country_code  = country_code,
              c.source_id     = source_ids[0],
              c.node_id       = node_ids[0],
              c.loc           = loc,
              c              += params
""")

[]

In [8]:
# consumers

conn.query("""
LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/iamvarol/blogposts/main/medium/europe_gas_network/data/IGGIELGNC3_Consumers.csv' as row
FIELDTERMINATOR ';'
WITH row.id as id,
     replace(row.name, '"','') as name,
     row.country_code as country_code,
     apoc.convert.fromJsonList(row.source_id) as source_ids,
     apoc.convert.fromJsonList(row.node_id) as node_ids,
     point({latitude:toFloat(row.lat), longitude:toFloat(row.long)}) as loc,
     apoc.convert.fromJsonMap(replace(row.param, 'None', 'null')) as params
MERGE(c:Consumer {id:id})
ON CREATE SET c.name          = name,
              c.country_code  = country_code,
              c.source_id     = source_ids[0],
              c.node_id       = node_ids[0],
              c.loc           = loc,
              c              += params
""")

[]

In [9]:
# LNGs

conn.query("""
LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/iamvarol/blogposts/main/medium/europe_gas_network/data/IGGIELGNC3_LNGs.csv' as row
FIELDTERMINATOR ';'
WITH row.id as id,
     replace(row.name, '"','') as name,
     row.country_code as country_code,
     apoc.convert.fromJsonList(row.source_id) as source_ids,
     apoc.convert.fromJsonList(row.node_id) as node_ids,
     point({latitude:toFloat(row.lat), longitude:toFloat(row.long)}) as loc,
     apoc.convert.fromJsonMap(replace(row.param, 'None', 'null')) as params
MERGE(l:LNG {id:id})
ON CREATE SET l.name          = name,
              l.country_code  = country_code,
              l.source_id     = source_ids,
              l.node_id       = node_ids[0],
              l.loc           = loc,
              l              += params
""")

[]

In [10]:
# storages

conn.query("""
LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/iamvarol/blogposts/main/medium/europe_gas_network/data/IGGIELGNC3_Storages.csv' as row
FIELDTERMINATOR ';'
WITH row.id as id,
     replace(row.name, '"','') as name,
     row.country_code as country_code,
     apoc.convert.fromJsonList(row.source_id) as source_ids,
     apoc.convert.fromJsonList(row.node_id) as node_ids,
     point({latitude:toFloat(row.lat), longitude:toFloat(row.long)}) as loc,
     apoc.convert.fromJsonMap(replace(row.param, 'None', 'null')) as params
MERGE(s:Storage {id:id})
ON CREATE SET s.name          = name,
              s.country_code  = country_code,
              s.source_id     = source_ids,
              s.node_id       = node_ids[0],
              s.loc           = loc,
              s              += params
""")

[]

In [11]:
# productions

conn.query("""
LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/iamvarol/blogposts/main/medium/europe_gas_network/data/IGGIELGNC3_Productions.csv' as row
FIELDTERMINATOR ';'
WITH row.id as id,
     replace(row.name, '"','') as name,
     row.country_code as country_code,
     apoc.convert.fromJsonList(row.source_id) as source_ids,
     apoc.convert.fromJsonList(row.node_id) as node_ids,
     point({latitude:toFloat(row.lat), longitude:toFloat(row.long)}) as loc,
     apoc.convert.fromJsonMap(replace(row.param, 'None', 'null')) as params
MERGE(p:Production {id:id})
ON CREATE SET p.name          = name,
              p.country_code  = country_code,
              p.source_id     = source_ids[0],
              p.node_id       = node_ids[0],
              p.loc           = loc,
              p              += params
""")

[]

In [12]:
# power plants

conn.query("""
LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/iamvarol/blogposts/main/medium/europe_gas_network/data/IGGIELGNC3_PowerPlants.csv' as row
FIELDTERMINATOR ';'
WITH row.id as id,
     replace(row.name, '"','') as name,
     row.country_code as country_code,
     apoc.convert.fromJsonList(row.source_id) as source_ids,
     apoc.convert.fromJsonList(row.node_id) as node_ids,
     point({latitude:toFloat(row.lat), longitude:toFloat(row.long)}) as loc,
     apoc.convert.fromJsonMap(replace(row.param, 'None', 'null')) as params
MERGE(pp:PowerPlant {id:id})
ON CREATE SET pp.name          = name,
              pp.country_code  = country_code,
              pp.source_id     = source_ids,
              pp.node_id       = node_ids[0],
              pp.loc           = loc,
              pp              += params
""")

[]

In [13]:
# nodes

conn.query("""
LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/iamvarol/blogposts/main/medium/europe_gas_network/data/IGGIELGNC3_Nodes.csv' as row
FIELDTERMINATOR ';'
WITH row.id as id,
     replace(row.name, '"','') as name,
     row.country_code as country_code,
     apoc.convert.fromJsonList(row.source_id) as source_ids,
     apoc.convert.fromJsonList(row.node_id) as node_ids,
     point({latitude:toFloat(row.lat), longitude:toFloat(row.long)}) as loc,
     apoc.convert.fromJsonMap(replace(row.param, 'None', 'null')) as params
MERGE(n:Node {id:id})
ON CREATE SET n.name          = name,
              n.country_code  = country_code,
              n.source_id     = source_ids[0],
              n.node_id       = node_ids[0],
              n.loc           = loc,
              n              += params
""")

[]

In [14]:
# pipe segments

conn.query("""
CALL apoc.periodic.iterate("
      LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/iamvarol/blogposts/main/medium/europe_gas_network/data/IGGIELGNC3_PipeSegments.csv' as row
FIELDTERMINATOR ';'
WITH row.id as id,
     row.name as name,
     apoc.convert.fromJsonList(row.country_code) as countries,
     apoc.convert.fromJsonList(row.node_id) as nodes,
     apoc.convert.fromJsonMap(replace(row.param, 'None', 'null')) as params
MATCH (start_p:Node {id:nodes[0]})
MATCH (end_p:Node {id:nodes[1]})
RETURN start_p, end_p, id, name, params, countries
","
      MERGE (start_p)-[pipe:PIPE {id:id}]->(end_p)
      ON CREATE SET pipe.name = name,
                    pipe += params
", {batchSize:5, parallel:true}
)
""")

[<Record batches=1306 total=6526 timeTaken=0 committedOperations=6516 failedOperations=10 failedBatches=2 retries=0 errorMessages={"ForsetiClient[transactionId=1524, clientId=2] can't acquire ExclusiveLock{owner=ForsetiClient[transactionId=1523, clientId=6]} on NODE_RELATIONSHIP_GROUP_DELETE(2556), because holders of that lock are waiting for ForsetiClient[transactionId=1524, clientId=2].\n Wait list:ExclusiveLock[\r\nClient[1523] waits for [ForsetiClient[transactionId=1524, clientId=2]]]": 1, "ForsetiClient[transactionId=2194, clientId=3] can't acquire ExclusiveLock{owner=ForsetiClient[transactionId=2190, clientId=5]} on NODE_RELATIONSHIP_GROUP_DELETE(5032), because holders of that lock are waiting for ForsetiClient[transactionId=2194, clientId=3].\n Wait list:ExclusiveLock[\r\nClient[2190] waits for [ForsetiClient[transactionId=2194, clientId=3],ForsetiClient[transactionId=-1, clientId=6]]]": 1} batch={'total': 1306, 'committed': 1304, 'failed': 2, 'errors': {"org.neo4j.kernel.Deadlo

## EDA

In [15]:
q = """
CALL db.labels() YIELD label
CALL apoc.cypher.run('MATCH (:`'+label+'`) RETURN count(*) as count',{}) YIELD value
RETURN label as nodes, value.count as nodeCount
ORDER BY nodeCount DESC
"""

df = pd.DataFrame([dict(_) for _ in conn.query(q)])
df

,nodes,nodeCount
0,Node,5009
1,Consumer,1357
2,PowerPlant,310
3,Storage,297
4,Compressor,248
5,BorderPoint,109
6,Production,104
7,LNG,32


In [16]:
q = '''
CALL db.relationshipTypes() YIELD relationshipType as type
CALL apoc.cypher.run('MATCH ()-[:`'+type+'`]->() RETURN count(*) as count',{}) YIELD value
RETURN type as relationship, value.count as relationshipCount
'''
df = pd.DataFrame([dict(_) for _ in conn.query(q)])
df

,relationship,relationshipCount
0,PIPE,6516


In [17]:
q = '''
CALL apoc.meta.stats()
'''
df = pd.DataFrame([dict(_) for _ in conn.query(q)])
df

,labelCount,relTypeCount,propertyKeyCount,nodeCount,relCount,labels,relTypes,relTypesCount,stats
0,8,1,83,7466,6516,"{'BorderPoint': 109, 'Storage': 297, 'Consumer': 1357, 'LNG': 32, 'PowerPlant': 310, 'Node': 5009, 'Production': 104, 'Compressor': 248}","{'()-[:PIPE]->(:Node)': 6516, '()-[:PIPE]->()': 6516, '(:Node)-[:PIPE]->()': 6516}",{'PIPE': 6516},"{'relTypeCount': 1, 'propertyKeyCount': 83, 'labelCount': 8, 'nodeCount': 7466, 'relCount': 6516, 'labels': {'BorderPoint': 109, 'Storage': 297, 'Consumer': 1357, 'LNG': 32, 'PowerPlant': 310, 'Node': 5009, 'Production': 104, 'Compressor': 248}, 'relTypes': {'()-[:PIPE]->(:Node)': 6516, '()-[:PIPE]->()': 6516, '(:Node)-[:PIPE]->()': 6516}}"


In [18]:
query="""
MATCH p=(start:Node)-[:PIPE*20]->(next)
WHERE not ((next)-[:PIPE]->()) AND start.id="INET_N_856"
RETURN DISTINCT [t in nodes(p) | t.id] as nodesPath, [t in nodes(p) | t.country_code] as countryPath
"""
df = pd.DataFrame([dict(_) for _ in conn.query(query)])
df

,nodesPath,countryPath
0,"[INET_N_856, SEQ_7_p, INET_N_1829, INET_N_329, SEQ_5085_p, INET_N_1805, SEQ_17253_p, SEQ_17262_p, SEQ_30_p, INET_N_933, SEQ_30_p, INET_N_933, LKD_N_96, LKD_N_516, LKD_N_517, LKD_N_518, LKD_N_549, LKD_N_518, LKD_N_521, SEQ_19620_p, INET_N_1637]","[PL, PL, PL, PL, PL, PL, PL, PL, PL, DE, PL, DE, DE, DE, DE, DE, DE, DE, DE, DE, DE]"
1,"[INET_N_856, SEQ_7_p, INET_N_1829, INET_N_329, SEQ_5085_p, INET_N_1805, SEQ_17253_p, SEQ_17262_p, SEQ_30_p, INET_N_933, SEQ_30_p, INET_N_933, LKD_N_96, LKD_N_516, LKD_N_517, LKD_N_518, LKD_N_549, LKD_N_518, LKD_N_521, LKD_N_524, N_2287_S_LMGN]","[PL, PL, PL, PL, PL, PL, PL, PL, PL, DE, PL, DE, DE, DE, DE, DE, DE, DE, DE, DE, DE]"
2,"[INET_N_856, SEQ_7_p, INET_N_1829, INET_N_329, SEQ_5085_p, INET_N_1805, SEQ_17253_p, SEQ_17262_p, SEQ_30_p, INET_N_933, SEQ_30_p, INET_N_933, LKD_N_96, LKD_N_516, LKD_N_517, LKD_N_518, LKD_N_549, LKD_N_518, LKD_N_521, LKD_N_530, INET_N_1300]","[PL, PL, PL, PL, PL, PL, PL, PL, PL, DE, PL, DE, DE, DE, DE, DE, DE, DE, DE, DE, DE]"
3,"[INET_N_856, SEQ_7_p, INET_N_1829, INET_N_329, SEQ_5085_p, INET_N_1805, SEQ_17253_p, SEQ_17262_p, SEQ_30_p, INET_N_933, SEQ_30_p, INET_N_933, LKD_N_96, LKD_N_516, LKD_N_517, LKD_N_518, LKD_N_549, LKD_N_518, LKD_N_521, LKD_N_530, NutsCons_447]","[PL, PL, PL, PL, PL, PL, PL, PL, PL, DE, PL, DE, DE, DE, DE, DE, DE, DE, DE, DE, DE]"
4,"[INET_N_856, SEQ_7_p, INET_N_1829, INET_N_329, SEQ_5085_p, INET_N_1805, SEQ_17253_p, SEQ_17262_p, SEQ_30_p, INET_N_933, SEQ_30_p, INET_N_933, LKD_N_96, LKD_N_516, LKD_N_517, LKD_N_518, LKD_N_521, LKD_N_524, LKD_N_525, NutsCons_435, INET_N_173]","[PL, PL, PL, PL, PL, PL, PL, PL, PL, DE, PL, DE, DE, DE, DE, DE, DE, DE, DE, DE, DE]"
5,"[INET_N_856, SEQ_7_p, INET_N_1829, INET_N_329, SEQ_5085_p, INET_N_1805, SEQ_17253_p, SEQ_17262_p, SEQ_30_p, INET_N_933, SEQ_30_p, INET_N_933, LKD_N_96, LKD_N_516, LKD_N_517, LKD_N_518, LKD_N_521, LKD_N_530, LKD_N_535, SEQ_19639_p, NutsCons_472]","[PL, PL, PL, PL, PL, PL, PL, PL, PL, DE, PL, DE, DE, DE, DE, DE, DE, DE, DE, DE, DE]"
6,"[INET_N_856, SEQ_7_p, INET_N_1829, INET_N_329, SEQ_5085_p, INET_N_1805, SEQ_17253_p, SEQ_17262_p, SEQ_30_p, INET_N_933, SEQ_30_p, INET_N_933, LKD_N_96, LKD_N_516, LKD_N_517, LKD_N_521, SEQ_19620_p, LKD_N_527, LKD_N_538, SEQ_19649_p, NutsCons_349]","[PL, PL, PL, PL, PL, PL, PL, PL, PL, DE, PL, DE, DE, DE, DE, DE, DE, DE, DE, DE, DE]"
7,"[INET_N_856, SEQ_7_p, INET_N_1829, INET_N_329, SEQ_5085_p, INET_N_1805, SEQ_17253_p, SEQ_17262_p, SEQ_30_p, INET_N_933, SEQ_30_p, INET_N_933, LKD_N_96, LKD_N_516, LKD_N_517, LKD_N_521, SEQ_19620_p, LKD_N_527, LKD_N_531, LKD_N_530, INET_N_1300]","[PL, PL, PL, PL, PL, PL, PL, PL, PL, DE, PL, DE, DE, DE, DE, DE, DE, DE, DE, DE, DE]"
8,"[INET_N_856, SEQ_7_p, INET_N_1829, INET_N_329, SEQ_5085_p, INET_N_1805, SEQ_17253_p, SEQ_17262_p, SEQ_30_p, INET_N_933, SEQ_30_p, INET_N_933, LKD_N_96, LKD_N_516, LKD_N_517, LKD_N_521, SEQ_19620_p, LKD_N_527, LKD_N_531, LKD_N_530, NutsCons_447]","[PL, PL, PL, PL, PL, PL, PL, PL, PL, DE, PL, DE, DE, DE, DE, DE, DE, DE, DE, DE, DE]"
9,"[INET_N_856, SEQ_7_p, INET_N_1829, INET_N_329, SEQ_5085_p, INET_N_1805, SEQ_17253_p, SEQ_17262_p, SEQ_30_p, INET_N_933, SEQ_30_p, INET_N_933, LKD_N_96, SEQ_18708_p, SEQ_18711_p, LKD_N_97, SEQ_18724_p, SEQ_18726_p, SEQ_18727_p, SEQ_18729_p, INET_N_1576]","[PL, PL, PL, PL, PL, PL, PL, PL, PL, DE, PL, DE, DE, DE, DE, DE, DE, DE, DE, DE, DE]"


## Graph Data Science

In [21]:
!pip install -q graphdatascience

In [24]:
import pandas as pd
from graphdatascience import GraphDataScience

uri  = 'bolt://localhost:7687'
pwd  = 'graph4va'
user = 'neo4j'
gds = GraphDataScience(uri, auth=("neo4j", pwd))

In [50]:
# dropping an existing graph
query = "CALL gds.graph.drop('pipelines') YIELD graphName;"
gds.run_cypher(query)

In [26]:
query = """CALL gds.graph.project(
                'pipelines',
                'Node',
                'PIPE'
            )
            YIELD
                graphName, nodeProjection, nodeCount, relationshipProjection, relationshipCount
"""

gds.run_cypher(query)

,graphName,nodeProjection,nodeCount,relationshipProjection,relationshipCount
0,pipelines,"{'Node': {'label': 'Node', 'properties': {}}}",5009,"{'PIPE': {'orientation': 'NATURAL', 'aggregation': 'DEFAULT', 'type': 'PIPE', 'properties': {}}}",6516


In [27]:
G, res = gds.graph.project(
    'pipes',
    'Node',
    "PIPE"
)
type(res)

pandas.core.series.Series

In [28]:
print(res)

nodeProjection            {'Node': {'label': 'Node', 'properties': {}}}                                                   
relationshipProjection    {'PIPE': {'orientation': 'NATURAL', 'aggregation': 'DEFAULT', 'type': 'PIPE', 'properties': {}}}
graphName                 pipes                                                                                           
nodeCount                 5009                                                                                            
relationshipCount         6516                                                                                            
projectMillis             28                                                                                              
Name: 0, dtype: object


In [29]:
G.node_count(), G.relationship_count()

(5009, 6516)

In [30]:
G.degree_distribution()

p99     5.000000 
min     0.000000 
max     12.000000
mean    1.300858 
p90     3.000000 
p50     1.000000 
p999    7.000000 
p95     3.000000 
p75     2.000000 
dtype: float64

In [31]:
res = gds.pageRank.stream(G)
res

,nodeId,score
0,2457,0.331688
1,2458,0.431934
2,2459,0.333572
3,2460,0.597695
4,2461,0.515261
...,...,...
5004,7461,0.411465
5005,7462,0.319784
5006,7463,0.267938
5007,7464,0.213750


In [32]:
# top 10 by page rank
res.sort_values(by='score', ascending=False)[:10]

,nodeId,score
2848,5305,4.861048
2852,5309,4.331384
514,2971,4.225036
1813,4270,3.832889
1708,4165,3.320917
1769,4226,3.236406
1776,4233,3.128835
1709,4166,2.935067
2779,5236,2.765317
2112,4569,2.691821


In [33]:
query = """
MATCH (n:Node)
WHERE ID(n) = 5305
RETURN n.id
"""
df = pd.DataFrame([dict(_) for _ in conn.query(query)])
df

Failed to write data to connection IPv4Address(('localhost', 7687)) (ResolvedIPv4Address(('127.0.0.1', 7687)))


Query failed: Failed to write data to connection IPv4Address(('localhost', 7687)) (ResolvedIPv4Address(('127.0.0.1', 7687)))


TypeError: 'NoneType' object is not iterable

In [34]:
# Centrality determined by node degree
query = """
MATCH (n:Node)
WITH n, SIZE((n)-[:PIPE]-(:Node)) AS degree
RETURN n.id AS id, n.country_code AS node_code, degree
ORDER BY degree DESC
"""
df = pd.DataFrame([dict(_) for _ in conn.query(query)])
df

,id,node_code,degree
0,LKD_N_38,DE,14
1,N_144_L_LMGN,GB,13
2,N_72_L_LMGN,RU,11
3,INET_N_1679,UA,11
4,LKD_N_363,DE,11
...,...,...,...
5004,INET_N_1408,IT,1
5005,NutsCons_1362,TR,1
5006,Prod_51,XX,1
5007,NutsCons_714,EE,1


In [35]:
# measure the nodes by indegree count
query = """MATCH (n:Node)
WITH n, SIZE((n)<-[:PIPE]-(:Node)) AS indegree
RETURN n.id AS id, n.country_code AS node_code, indegree
ORDER BY indegree DESC"""

df = pd.DataFrame([dict(_) for _ in conn.query(query)])
df

,id,node_code,indegree
0,INET_N_1071,NL,7
1,INET_N_883,UA,7
2,LKD_N_40,DE,7
3,LKD_N_77,DE,6
4,LKD_N_132,DE,6
...,...,...,...
5004,N_2495_S_LMGN,FR,0
5005,INET_N_863,UA,0
5006,SEQ_1305__M_LMGN,HU,0
5007,N_671_S_LMGN,PT,0


In [36]:
# writing the page ranks of the nodes
q= """CALL gds.pageRank.write('pipes',
    {
        writeProperty: 'pagerank'
    }
)
YIELD nodePropertiesWritten, ranIterations"""
gds.run_cypher(q)

,nodePropertiesWritten,ranIterations
0,5009,20


In [37]:
q = """MATCH (n:Node)<-[r:PIPE]-(:Node)
RETURN n.id AS id, n.country_code AS countryCode, round(n.pagerank,2) AS pagerank, round(sum(r.max_cap_M_m3_per_d),2) as maximumAnnualGasVolume
ORDER BY pagerank DESC, id ASC
LIMIT 10"""
df = pd.DataFrame([dict(_) for _ in conn.query(q)])
df

,id,countryCode,pagerank,maximumAnnualGasVolume
0,LKD_N_40,DE,4.86,195.52
1,LKD_N_44,DE,4.33,54.79
2,N_183_M_LMGN,MD,4.23,82.19
3,SEQ_1813__M_LMGN,MD,3.83,54.79
4,SEQ_7563__L_LMGN,XX,3.32,82.19
5,N_32_M_LMGN,RU,3.24,54.79
6,N_85_M_LMGN,RU,3.13,54.79
7,SEQ_7577__L_LMGN,XX,2.94,27.40
8,SEQ_2409_p,AZ,2.77,54.79
9,INET_N_408,UA,2.69,164.38


## Degree Centrality

In [38]:
query = """
CALL gds.graph.project(
  'degreeCentralityPipes',
  'Node',
  {
    PIPE: {
      orientation: "UNDIRECTED"
    }
  }
)
"""
gds.run_cypher(query)

,nodeProjection,relationshipProjection,graphName,nodeCount,relationshipCount,projectMillis
0,"{'Node': {'label': 'Node', 'properties': {}}}","{'PIPE': {'orientation': 'UNDIRECTED', 'aggregation': 'DEFAULT', 'type': 'PIPE', 'properties': {}}}",degreeCentralityPipes,5009,13032,15


In [39]:
query = """
CALL gds.degree.stream('degreeCentralityPipes')
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).id AS id, score AS degree
ORDER BY degree DESC, id ASC
LIMIT 10
"""
gds.run_cypher(query)

,id,degree
0,LKD_N_38,14.0
1,N_144_L_LMGN,13.0
2,INET_N_1679,11.0
3,INET_N_408,11.0
4,LKD_N_147,11.0
5,LKD_N_363,11.0
6,N_72_L_LMGN,11.0
7,INET_N_1071,10.0
8,INET_N_1118,10.0
9,INET_N_1579,10.0


## Betweenness Centrality

In [40]:
query = "CALL gds.graph.project('betweennessPipes', 'Node', 'PIPE')"
gds.run_cypher(query)

,nodeProjection,relationshipProjection,graphName,nodeCount,relationshipCount,projectMillis
0,"{'Node': {'label': 'Node', 'properties': {}}}","{'PIPE': {'orientation': 'NATURAL', 'aggregation': 'DEFAULT', 'type': 'PIPE', 'properties': {}}}",betweennessPipes,5009,6516,10


In [41]:
query = """
CALL gds.betweenness.write.estimate('betweennessPipes', { writeProperty: 'betweenness' })
YIELD nodeCount, relationshipCount, bytesMin, bytesMax, requiredMemory
"""

gds.run_cypher(query)

,nodeCount,relationshipCount,bytesMin,bytesMax,requiredMemory
0,5009,6516,1404224,1404224,1371 KiB


In [42]:
query = """
CALL gds.betweenness.stats('betweennessPipes')
YIELD centralityDistribution
RETURN centralityDistribution.min AS minimumScore, centralityDistribution.mean AS meanScore, centralityDistribution.max AS maxScore
"""
gds.run_cypher(query)

,minimumScore,meanScore,maxScore
0,0.0,17748.48583,863607.999992


In [43]:
query = """
CALL gds.betweenness.stream('betweennessPipes')
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).id AS id, round(score,2) as score
ORDER BY score DESC, id ASC
LIMIT 10
"""

gds.run_cypher(query)

,id,score
0,INET_N_984,863605.26
1,LKD_N_38,606942.11
2,LKD_N_492,600710.37
3,INET_N_229,597845.66
4,SEQ_19574_p,597400.37
5,SEQ_19575_p,596701.37
6,LKD_N_491,596621.84
7,LKD_N_493,596002.37
8,SEQ_719__L_LMGN,594105.73
9,LKD_N_165,594088.60


In [44]:
query = """
CALL gds.betweenness.stream('betweennessPipes')
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).id AS id, round(score,2) as score
ORDER BY score DESC, id ASC
"""

betweenness_df = gds.run_cypher(query)

In [45]:
betweenness_df.head(20)

,id,score
0,INET_N_984,863605.26
1,LKD_N_38,606942.11
2,LKD_N_492,600710.37
3,INET_N_229,597845.66
4,SEQ_19574_p,597400.37
5,SEQ_19575_p,596701.37
6,LKD_N_491,596621.84
7,LKD_N_493,596002.37
8,SEQ_719__L_LMGN,594105.73
9,LKD_N_165,594088.60


## Louvain

In [46]:
query = """
CALL gds.louvain.stream('pipes')
YIELD nodeId, communityId
RETURN
	communityId,
    SIZE(COLLECT(gds.util.asNode(nodeId).id)) AS number_of_nodes,
	COLLECT(gds.util.asNode(nodeId).id) AS ids,
    COLLECT(gds.util.asNode(nodeId).country_code) AS country_codes
ORDER BY number_of_nodes DESC, communityId;
"""
clusters_df = gds.run_cypher(query)
clusters_df.head(10)

,communityId,number_of_nodes,ids,country_codes
0,3480,195,"[INET_N_817, SEQ_7777_p, INET_N_755, INET_N_929, SEQ_667_p, SEQ_7802_p, SEQ_678_p, SEQ_7811_p, SEQ_7829_p, SEQ_7859_p, SEQ_7882_p, SEQ_702_p, INET_N_1490, INET_N_1689, INET_N_1188, SEQ_8530_p, INET_N_1690, SEQ_8615_p, INET_N_979, SEQ_8671_p, INET_N_402, SEQ_8753_p, SEQ_831_p, SEQ_8881_p, INET_N_1691, INET_N_885, INET_N_1814, INET_N_1555, SEQ_9854_p, SEQ_9876_p, SEQ_9896_p, INET_N_1280, INET_N_839, INET_N_64, INET_N_883, INET_N_139, SEQ_11191_p, INET_N_717, SEQ_11233_p, INET_N_1758, SEQ_1040_p, INET_N_780, SEQ_11307_p, SEQ_1051_p, SEQ_11529_p, INET_N_1598, SEQ_11608_p, INET_N_1579, INET_N_1187, INET_N_1589, SEQ_1160_p, SEQ_12580_p, N_168_L_LMGN, N_173_L_LMGN, SEQ_2829__L_LMGN, N_126_L_LMGN, SEQ_5383__M_LMGN, N_183_M_LMGN, N_10_L_LMGN, SEQ_3791__L_LMGN, SEQ_3793__L_LMGN, N_1604_S_LMGN, SEQ_1319__M_LMGN, SEQ_4715_p, N_312_M_LMGN, SEQ_5601__M_LMGN, SEQ_5597__M_LMGN, SEQ_7047__L_LMGN, SEQ_1797__M_LMGN, INET_N_742, SEQ_15043__S_LMGN, N_393_S_LMGN, N_711_S_LMGN, N_175_L_LMGN, SEQ_1229__L_LMGN, SEQ_3975__L_LMGN, Prod_3, N_477_M_LMGN, N_145_M_LMGN, SEQ_7009__L_LMGN, SEQ_9097__M_LMGN, N_2585_M_LMGN, SEQ_1235__L_LMGN, SEQ_5169__M_LMGN, SEQ_1237__L_LMGN, SEQ_4395__M_LMGN, SEQ_7005__L_LMGN, SEQ_7001__L_LMGN, SEQ_6995__L_LMGN, SEQ_7781_p, N_72_L_LMGN, INET_N_893, SEQ_3079__M_LMGN, N_472_M_LMGN, N_594_M_LMGN, SEQ_1255__L_LMGN, SEQ_9386_p, SEQ_3813__L_LMGN, SEQ_3817__L_LMGN, N_182_L_LMGN, ...]","[UA, UA, UA, UA, UA, UA, UA, UA, UA, UA, UA, UA, UA, UA, UA, UA, UA, UA, UA, UA, UA, UA, UA, UA, UA, UA, UA, UA, UA, UA, UA, UA, UA, UA, UA, UA, UA, UA, UA, UA, UA, UA, UA, UA, UA, UA, UA, UA, UA, UA, UA, UA, UA, UA, UA, UA, MD, MD, BY, BY, BY, BY, UA, UA, UA, UA, UA, MD, MD, RO, RO, UA, UA, UA, BY, UA, UA, UA, UA, RO, UA, UA, BY, BY, BY, UA, RO, RO, RO, RO, RU, RU, UA, RU, RU, BY, BY, BY, BY, RU, ...]"
1,4496,186,"[N_61_L_LMGN, N_163_L_LMGN, N_150_L_LMGN, SEQ_3245__L_LMGN, N_98_L_LMGN, SEQ_2157__L_LMGN, SEQ_8233__M_LMGN, N_1302_S_LMGN, SEQ_9997__M_LMGN, SEQ_1900_p, N_811_M_LMGN, SEQ_9973__M_LMGN, SEQ_9971__M_LMGN, N_2320_S_LMGN, SEQ_12551__S_LMGN, SEQ_8005__L_LMGN, SEQ_7999__L_LMGN, N_117_L_LMGN, SEQ_2599__L_LMGN, SEQ_2366_p, SEQ_17843__S_LMGN, N_2120_S_LMGN, SEQ_2582_p, SEQ_2586_p, SEQ_2593__L_LMGN, SEQ_2597__L_LMGN, Stor_EU_33, SEQ_2153__L_LMGN, N_97_L_LMGN, SEQ_21287__S_LMGN, SEQ_2577__L_LMGN, SEQ_2575__L_LMGN, SEQ_3323__M_LMGN, N_341_M_LMGN, SEQ_4533__M_LMGN, N_434_M_LMGN, SEQ_3713__L_LMGN, SEQ_3707__L_LMGN, SEQ_3705__L_LMGN, N_145_L_LMGN, SEQ_3189__L_LMGN, N_146_L_LMGN, SEQ_4207__L_LMGN, SEQ_4209__L_LMGN, SEQ_4215__L_LMGN, SEQ_4219__L_LMGN, SEQ_4223__L_LMGN, N_56_L_LMGN, SEQ_1295__L_LMGN, SEQ_9995__M_LMGN, SEQ_1365__L_LMGN, SEQ_4975_p, SEQ_7017__M_LMGN, N_118_L_LMGN, N_1348_S_LMGN, N_2238_S_LMGN, N_2237_S_LMGN, N_2239_S_LMGN, SEQ_19623__S_LMGN, N_186_L_LMGN, N_57_L_LMGN, SEQ_1317__L_LMGN, SEQ_1311__L_LMGN, N_147_L_LMGN, SEQ_6599__L_LMGN, SEQ_5273__L_LMGN, SEQ_5285__L_LMGN, SEQ_1291__L_LMGN, SEQ_1387__L_LMGN, N_1366_S_LMGN, N_1352_S_LMGN, N_1354_S_LMGN, SEQ_1361__L_LMGN, N_60_L_LMGN, SEQ_6514_p, N_210_L_LMGN, SEQ_7497__L_LMGN, SEQ_7493__L_LMGN, SEQ_6524_p, SEQ_15277__S_LMGN, N_1729_S_LMGN, N_2000_S_LMGN, N_2001_S_LMGN, SEQ_4371__L_LMGN, SEQ_4367__L_LMGN, SEQ_1371__L_LMGN, SEQ_1375__L_LMGN, SEQ_6895_p, SEQ_10825__M_LMGN, N_1158_S_LMGN, N_1356_S_LMGN, N_1855_S_LMGN, SEQ_8134_p, SEQ_8140_p, SEQ_1307__L_LMGN, SEQ_8334_p, SEQ_8225__M_LMGN, SEQ_3335__M_LMGN, N_342_M_LMGN, N_1954_S_LMGN, ...]","[FR, FR, FR, FR, FR, FR, FR, FR, FR, FR, FR, FR, FR, FR, FR, FR, FR, FR, FR, FR, FR, FR, FR, FR, FR, FR, FR, FR, FR, FR, FR, FR, FR, FR, FR, FR, FR, FR, FR, FR, FR, FR, FR, FR, FR, FR, FR, FR, FR, FR, FR, FR, FR, FR, FR, FR, FR, FR, FR, FR, FR, FR, FR, FR, FR, FR, FR, FR, FR, FR, FR, FR, FR, FR, FR, FR, FR, FR, FR, FR, FR, FR, FR, FR, FR, FR, FR, FR, FR, FR, FR, FR, FR, FR, FR, FR, FR, FR, FR, FR, ...]"
2,4971,175,"[INET_N_1071, N_271_NS_LMGN, N_281_NS_LMGN, N_53_NS_LMGN, N_52_NS_LMGN, N_39_NS_LMGN, N_152

In [47]:
clusters_df.tail(10)

,communityId,number_of_nodes,ids,country_codes
212,4697,1,[INET_N_1],[BE]
213,4710,1,[INET_N_947],[DE]
214,4732,1,[INET_N_241],[SI]
215,4740,1,[INET_N_270],[GB]
216,4803,1,[INET_N_1454],[IT]
217,4806,1,[INET_N_1499],[IT]
218,4922,1,[INET_N_900],[SK]
219,4929,1,[NutsCons_1515],[XX]
220,4956,1,[NutsCons_597],[XX]
221,4973,1,[INET_N_522],[XX]


## Path Finding

In [48]:
query = """
CALL gds.graph.project(
    'routes-weighted',
    'Node',
    'PIPE',
        {
            relationshipProperties: 'length_km'
        }
)
"""

gds.run_cypher(query)

,nodeProjection,relationshipProjection,graphName,nodeCount,relationshipCount,projectMillis
0,"{'Node': {'label': 'Node', 'properties': {}}}","{'PIPE': {'orientation': 'NATURAL', 'aggregation': 'DEFAULT', 'type': 'PIPE', 'properties': {'length_km': {'defaultValue': None, 'property': 'length_km', 'aggregation': 'DEFAULT'}}}}",routes-weighted,5009,6516,43


In [49]:
query = """
MATCH (source:Node {id: 'INET_N_856'}), (target:Node {id: 'NutsCons_1003'})
CALL gds.shortestPath.dijkstra.stream('routes-weighted', {
    sourceNode: source,
    targetNode: target,
    relationshipWeightProperty: 'length_km'
})
YIELD index, sourceNode, targetNode, totalCost, nodeIds, costs, path
RETURN
    index,
    gds.util.asNode(sourceNode).id AS sourceNodeName,
    gds.util.asNode(targetNode).id AS targetNodeName,
    totalCost as totalDistance,
    [nodeId IN nodeIds | gds.util.asNode(nodeId).id] AS nodeIDs,
    costs as distances
ORDER BY index
"""
gds.run_cypher(query)

,index,sourceNodeName,targetNodeName,totalDistance,nodeIDs,distances
0,0,INET_N_856,NutsCons_1003,649.707706,"[INET_N_856, SEQ_7_p, INET_N_1829, INET_N_329, N_409_S_LMGN, SEQ_9701__M_LMGN, SEQ_9149__S_LMGN, SEQ_9139__S_LMGN, SEQ_9127__S_LMGN, N_486_M_LMGN, SEQ_7715__M_LMGN, SEQ_7723__M_LMGN, SEQ_7731__M_LMGN, N_688_M_LMGN, SEQ_7739__L_LMGN, SEQ_7743__L_LMGN, SEQ_7747__L_LMGN, SEQ_7749__L_LMGN, NutsCons_1003]","[0.0, 55.420951, 110.841205, 220.16349100000002, 253.03525000000002, 260.771973, 289.15599, 317.347467, 352.397237, 372.523689, 398.64748099999997, 448.01613399999997, 487.93127499999997, 540.4601309999999, 569.220877, 597.2742549999999, 630.323322, 638.421699, 649.707706]"
